In [ ]:
from scipy.stats import kurtosis, skew

def readdata_MT5(market,tf,barre,path):
    from datetime import datetime
    import MetaTrader5 as mt5
    import pandas as pd
    

    while True:

        if not mt5.initialize(path):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        mk=str(market)
        TF="mt5.TIMEFRAME_"+str(tf)
        bar=barre

        rates = mt5.copy_rates_from_pos(mk, eval(TF), 0, bar) 
    
        try:
            rates_frame = pd.DataFrame(rates)
            rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
            rates_frame=rates_frame[["time","open","high","low","close","real_volume"]]
            rates_frame.columns=(["time","open","high","low","close","volume"])
            rates_frame=rates_frame.set_index(["time"])
            rates_frame["dayofweek"] = rates_frame.index.dayofweek
            rates_frame["day"] = rates_frame.index.day
            rates_frame["month"] = rates_frame.index.month
            rates_frame["year"] = rates_frame.index.year
            rates_frame["dayofyear"] = rates_frame.index.dayofyear
            rates_frame["quarter"] = rates_frame.index.quarter
            rates_frame["hour"] = rates_frame.index.hour
            rates_frame["minute"] = rates_frame.index.minute
            rates_frame["daily_open"] = daily_open(rates_frame,1)
            rates_frame["daily_high"] = daily_high(rates_frame,1)
            rates_frame["daily_low"] = daily_low(rates_frame,1)
            rates_frame["daily_close"] = daily_close(rates_frame,1)
            rates_frame["AVGPRICE"]=ta.AVGPRICE(rates_frame.open, rates_frame.high, rates_frame.low, rates_frame.close)
            rates_frame["MEDPRICE"]=ta.MEDPRICE(rates_frame.high, rates_frame.low)
            rates_frame["TYPPRICE"]=ta.TYPPRICE(rates_frame.high, rates_frame.low, rates_frame.close)
            rates_frame["WCLPRICE"]=ta.WCLPRICE(rates_frame.high, rates_frame.low, rates_frame.close)
            
            if rates_frame is not None:
                return rates_frame
                break
        except KeyError:
            continue 
            
def tick_info(simbolo):
    import MetaTrader5 as mt5
    tick_info = mt5.symbol_info(simbolo)
    tick_info = float(tick_info.trade_tick_size)
    return tick_info

def Big_Point_Values(simbolo,path):
    import MetaTrader5 as mt5
    if not mt5.initialize(path):
        print("initialize() failed, error code =",mt5.last_error())
        quit()    
    
    symbol_info_dict = mt5.symbol_info(simbolo)._asdict()
    #display(symbol_info_dict)
    bpv=symbol_info_dict['trade_tick_value']  /symbol_info_dict['point']
    return bpv    
            
def profitto(operations):
    return operations.sum()

def EV(a):
    PW=float(a.PctWin)/100
    PL=float(100-float(a.PctWin))/100
    AW=int(a.gross_profit)
    AL=abs(int(a.gross_loss))
    EV=(PW*AW)-(PL*AL)
    return EV

def performance_report_gen(tradelist):
    res=pd.DataFrame()
    d = {"Profit": profitto(tradelist),
         "MaxDD" : max_draw_down(tradelist),
         "Operations":operation_number(tradelist),
         "AverageTrade": avg_trade(tradelist),
         "ProfitFactor": profit_factor(tradelist),
         "PctWin": percent_win(tradelist),
         "KestnerRatio": kestner_ratio(tradelist),
         "STD_Daily" : tradelist.resample("D").sum().std(),
         "Skew" : skew(tradelist.values),
         "Reward_Risk_Ratio": reward_risk_ratio(tradelist)}
    
    res=pd.DataFrame(data=d,index=[0])
    return res 

def test_time_frame(tf):
    tf=tf.lower()
    ls_tf=["1min","5min","15min","30min","60min","120min","240min","480min","1380min","1440min"]
    test_ok=0
    for e in ls_tf:
        if e == tf:
            tf_ok=e
            test_ok=1
            break
    if test_ok == 0:
            print("ERRORE - Solo time frame a: ",ls_tf," in ingresso")
    else:
        tf_ok=tf_ok.split("min")[0]
        return int(tf_ok)
    
def resample_standard_session(original_tf,resample_tf,dataset):
    
    original_tf=test_time_frame(original_tf)
    resample_tf=test_time_frame(resample_tf)
    df_0=dataset.copy()
    
    df_0.index=df_0.index - pd.DateOffset(minutes=original_tf)
    df_resample=df_0.resample(str(resample_tf)+'Min').agg({'open' : 'first', 'high' : 'max', 'low' : 'min', 'close' : 'last','volume':'sum'})
    
    if resample_tf == "1440" :
        df_resample = df_resample
    else:
        df_resample.index=df_resample.index + pd.DateOffset(minutes=resample_tf)
        
    df_resample=df_resample.dropna()
    return df_resample

def load_data_intraday_fast(filename):
    data=pd.read_csv(filename,engine="pyarrow")
    data.set_index(["date_time"],inplace=True)
    data.index=pd.to_datetime(data.index)
    return data

def resample_custom_session(original_tf,resample_tf,dataset,start_time,end_time):
    
    original_tf=test_time_frame(original_tf)
    resample_tf=test_time_frame(resample_tf)
    df_0=dataset.copy()
    
    df_0.index=df_0.index - pd.DateOffset(minutes=original_tf)
    
    test_start_time=0
    if len(start_time) == 4:
        st_h=start_time[:2]
        st_m=start_time[2:]
        test_start_time=1
        
    test_end_time=0
    if len(end_time) == 4:
        en_h=end_time[:2]
        en_m=end_time[2:]
        test_end_time=1

    if (test_start_time==0)|(test_end_time==0):
        print("ERRORE - controlla gli orari di start & end")
    else:

        dati=df_0.loc[(df_0.index.time>=datetime.time(int(st_h), int(st_m)))&(df_0.index.time<=datetime.time(int(en_h), int(en_m)))]
        dati= dati.resample(str(resample_tf)+'Min',origin=st_h+":"+st_m+':00').agg({'open': 'first','high': 'max','low':'min','close':'last','volume':'sum'}).dropna()
        dati.index=dati.index + pd.DateOffset(minutes=resample_tf)
        
        test_h=sorted(list(set(dati.index.time)))
        
        time1 = datetime.time(int(en_h),int(en_m))
        timedelta = datetime.timedelta(minutes=original_tf)
        tmp_datetime = datetime.datetime.combine(datetime.date(1, 1, 1), time1)
        time2 = (tmp_datetime - timedelta).time()
        
        dati1=df_0.loc[(df_0.index.time>=test_h[-2])&(df_0.index.time<=time2)]
        dati1= dati1.resample("D").agg({'open': 'first','high': 'max','low':'min','close':'last','volume':'sum'}).dropna()
        dati1.index=dati1.index + pd.DateOffset(hours=int(en_h),minutes=int(en_m))

        custom_df=dati.loc[dati.index.time!=test_h[-1]].dropna()
        custom_df=pd.concat([custom_df,dati1])
        custom_df=custom_df.sort_index(ascending=True)
        return custom_df
    
def carica_storico(file_name,uct_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,noise,pct_noise):
    import os
    import talib as ta
    
    os.chdir(dir_history)
    data = load_data_intraday_fast(file_name)  
    print("Caricato storico")
    data = data.sort_index(ascending=True)
    data.index = data.index.tz_localize('Etc/Zulu')
    data.index = data.index.tz_convert(uct_offset )
    data.index = data.index.tz_localize(None)
    data = data[data!=0]
    #data = data.fillna(method="ffill")
    data = data.iloc[1:-1]
    print("Resample dei dati")
    if type_session == 1:
        dataset=resample_custom_session("5min",resample_tf ,data,custom_session_start,custom_session_stop) 
    if type_session == 0:    
        dataset=resample_standard_session("5min",resample_tf,data)
        
    if noise == 1:
        print("AGGIUNGO RUMORE")
        dataset=add_noise(dataset.open, dataset.high, dataset.low, dataset.close,dataset.volume, pct_noise)

    dataset = dataset.loc[IS:OOS]
    dataset["dayofweek"] = dataset.index.dayofweek
    dataset["day"] = dataset.index.day
    dataset["month"] = dataset.index.month
    dataset["year"] = dataset.index.year
    dataset["dayofyear"] = dataset.index.dayofyear
    dataset["week"] = dataset.index.week
    dataset["hour"] = dataset.index.hour
    dataset["minute"] = dataset.index.minute
    dataset["daily_open"] = daily_open(dataset,1)
    dataset["daily_high"] = daily_high(dataset,1)
    dataset["daily_low"] = daily_low(dataset,1)
    dataset["daily_close"] = daily_close(dataset,1)
    dataset["AVGPRICE"]=ta.AVGPRICE(dataset.open, dataset.high, dataset.low, dataset.close)
    dataset["MEDPRICE"]=ta.MEDPRICE(dataset.high, dataset.low)
    dataset["TYPPRICE"]=ta.TYPPRICE(dataset.high, dataset.low, dataset.close)
    dataset["WCLPRICE"]=ta.WCLPRICE(dataset.high, dataset.low, dataset.close)
    history=dataset.copy()
    
    print("Carico indicatori")

    
    import legitindicators

    #data_filter=dataset.iloc[:].copy()
    #for e in filter.columns:
        #data_filter[e]= legitindicators.super_smoother(filter[e], 10)
    
    #import talib as ta
    #ATR = ta.ATR(dataset.high,dataset.low,dataset.close,5)
    #ATR = ATR.fillna(0)
    #dataATR=dataset.copy()
    #for e in dataATR.columns:
        #dataATR[e]=dataATR[e]/ATR
        
    applica_indicatori=apply_indicator(dataset)
    dataset=applica_indicatori[0]
    dataset_ind = dataset.iloc[:,21:].copy()


    rules=dataset.iloc[:,21:].T.values
    rule_formulas=np.arange(len(rules))
    
    history.tail(10)
    
    history.close.plot(figsize=(20,10),title=SIMBOLO+"_"+RESAMPLE_TF)
    print("Fatto")
    return dataset , dataset_ind , history , rules, rule_formulas



In [ ]:
import random
def randrange_float(start, stop, step):
    return random.randint(0, int((stop - start) / step)) * step + start

In [ ]:
def environment(simbolo,type_session,resample_tf,start_date,end_date):
               
    file_name = check_history_name(simbolo,dir_history)

    IS = start_date
    OOS = end_date

    bigpointvalue=valori[valori.Physical==simbolo].BigPointValue.values[0]
    tick=valori[valori.Physical==simbolo].Tick.values[0]
    utc_offset = valori[valori.Physical==simbolo].Exchange_Time.values[0]
    custom_session_start = "0"+str(valori[valori.Physical==simbolo]["Custom Open"].values[0])
    custom_session_stop = str(valori[valori.Physical==simbolo]["Custom Close"].values[0])
    costi=valori[valori.Physical==simbolo].Cost.values[0]

    dataset , dataset_ind , history , rules, rule_formulas = carica_storico(file_name,utc_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,0,0)
    
    return dataset , dataset_ind , history , rules, rule_formulas , bigpointvalue , tick , utc_offset , custom_session_start , custom_session_stop , costi 



In [1]:
def OPEN(dataset):
    return dataset["open"]
def HIGH(dataset):
    return dataset["high"]
def LOW(dataset):
    return dataset["low"]
def CLOSE(dataset):
    return dataset["close"]

def High_Close(serie):
    return (serie.iloc[:,1] - serie.iloc[:,3])
def High_Open(serie):
    return (serie.iloc[:,1] - serie.iloc[:,0])
def Low_Close(serie):
    return (serie.iloc[:,2] - serie.iloc[:,3])
def Low_Open(serie):
    return (serie.iloc[:,2] - serie.iloc[:,0])

def avg_true_range(dataframe, period):
    dataframe["M1"] = dataframe.high - dataframe.low
    dataframe["M2"] = abs(dataframe.high - dataframe.low.shift(1)).fillna(0)
    dataframe["M3"] = abs(dataframe.low - dataframe.close.shift(1)).fillna(0)
    dataframe["Max"] = dataframe[["M1", "M2", "M3"]].max(axis = 1)
    dataframe["MeanMax"] = dataframe["Max"].rolling(period).mean()
    return dataframe.MeanMax.fillna(0)

def RSI(series, period):
    """
    Function to calculate the Relative Strength Index of a close serie
    """
    df = pd.DataFrame(series, index = series.index)
    df["chg"] = series.diff(1)
    df["gain"] = np.where(df.chg > 0, 1, 0)
    df["loss"] = np.where(df.chg <= 0, 1, 0)
    df["avg_gain"] = df.gain.rolling(period).sum() / period * 100
    df["avg_loss"] = df.loss.rolling(period).sum() / period * 100
    rs = abs(df["avg_gain"] / df["avg_loss"])
    rsi = 100 - (100 / (1 + rs))
    return rsi

def MFI(df, period):
    """
    Function to calculate the Money Flow Index of a price serie (OHLCV)
    """
    df["typical_price"] = (df.iloc[:,1] + df.iloc[:,2] + df.iloc[:,3]) / 3
    df["raw_money_flow"] = df.typical_price * df.iloc[:,4]
    df["chg"] = df.raw_money_flow.diff(1)
    df["pos_money_flow"] = np.where(df.chg > 0,1,0)
    df["neg_money_flow"] = np.where(df.chg <= 0,1,0)
    df["avg_gain"] = df.pos_money_flow.rolling(period).sum() / period * 100
    df["avg_loss"] = df.neg_money_flow.rolling(period).sum() / period * 100
    mfr = abs(df["avg_gain"] / df["avg_loss"])
    mfi = 100 - (100 / (1 + mfr))
    return mfi

def BollingerBand(serie,period,multiplier,upper_lower):
    if upper_lower == 0:
        return serie.rolling(period).mean() + multiplier * serie.rolling(period).std()
    if upper_lower == 1:
        return serie.rolling(period).mean() - multiplier - serie.rolling(period).std()

def ROC(serie,period):
    return ((serie / serie.shift(period)) - 1) * 100

def Momentum(serie,period):
    return (serie - serie.shift(period))

def SMA(serie,period):
    return serie.rolling(period).mean()

def Range(serie):
    """
    Function to calculate the Range of a price serie (OHLCV)
    """
    return (serie.iloc[:,1] - serie.iloc[:,2])

def Body(serie):
    """
    Function to calculate the Body of a price serie (OHLCV)
    """
    return (serie.iloc[:,3] - serie.iloc[:,0])

def AvgPrice(serie):
    """
    Function to calculate the AvgPrice of a price serie (OHLCV)
    """
    return (serie.iloc[:,0] + serie.iloc[:,1] + serie.iloc[:,2] + serie.iloc[:,3]) / 4

def MedPrice(serie):
    """
    Function to calculate the MedPrice of a price serie (OHLCV)
    """
    return (serie.iloc[:,1] + serie.iloc[:,2]) / 2

def MedBodyPrice(serie):
    """
    Function to calculate the MedBodyPrice of a price serie (OHLCV)
    """
    return (serie.iloc[:,0] + serie.iloc[:,3]) / 2

def Blastoff(serie):
    return abs(serie.iloc[:,0] - serie.iloc[:,3]) / (serie.iloc[:,1] - serie.iloc[:,2])

def OpenToLowLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.open) - np.log(data.low))

def OpenToAvgPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    return pd.Series(np.log(data.open) - np.log(data.avgprice))

def OpenToMedPriceLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.open) - np.log(data.medprice))

def OpenToOpenLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.open) - np.log(data.open.shift(1)))

def OpenToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.open) - np.log(data.medbodyprice))

def CloseToMedPriceLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.close) - np.log(data.medprice))

def CloseToCloseLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.close) - np.log(data.close.shift(1)))

def CloseToLowLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.close) - np.log(data.low))

def CloseToAvgPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    return pd.Series(np.log(data.close) - np.log(data.avgprice))

def CloseToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.close) - np.log(data.medbodyprice))

def HighToCloseLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.high) - np.log(data.close))

def HighToOpenLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.high) - np.log(data.open))

def HighToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.high) - np.log(data.medbodyprice))

def HighToMedPriceLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.high) - np.log(data.medprice))

def HighToAvgPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    return pd.Series(np.log(data.high) - np.log(data.avgprice))

def HighToHighLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.high) - np.log(data.high.shift(1)))

def LowToLowLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.low) - np.log(data.low.shift(1)))

def AvgPriceToMedPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.avgprice) - np.log(data.medprice))

def AvgPriceToLowLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    return pd.Series(np.log(data.avgprice) - np.log(data.low))

def AvgPriceToAvgPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    return pd.Series(np.log(data.avgprice) - np.log(data.avgprice.shift(1)))

def AvgPriceToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["avgprice"] = data.iloc[:,:4].mean(axis = 1)
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.avgprice) - np.log(data.medbodyprice))

def MedPriceToLowLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.medprice) - np.log(data.low))

def MedBodyPriceToLowLog(dataset):
    data = dataset.copy()
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.medbodyprice) - np.log(data.low))

def MedPriceToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.medprice) - np.log(data.medbodyprice))

def MedPriceToMedPriceLog(dataset):
    data = dataset.copy()
    data["medprice"] = (data.high + data.low) / 2
    return pd.Series(np.log(data.medprice) - np.log(data.medprice.shift(1)))

def MedBodyPriceToMedBodyPriceLog(dataset):
    data = dataset.copy()
    data["medbodyprice"] = (data.open + data.close) / 2
    return pd.Series(np.log(data.medbodyprice) - np.log(data.medbodyprice).shift(1))

def RangeLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.high) - np.log(data.low))

def BodyLog(dataset):
    data = dataset.copy()
    return pd.Series(np.log(data.close) - np.log(data.open))

def BodyToRangeLog(dataset):
    data = dataset.copy()
    data["body"] = (data.close - data.open) 
    data["range"] = (data.high - data.low) 
    return pd.Series(np.log(data.body) - np.log(data.range))

def daily_high(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["day"] != df["day"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["high"].groupby(df["grouper"]).max().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_high" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def weekly_high(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["week"] != df["week"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["high"].groupby(df["grouper"]).max().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_high" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]
    
def daily_low(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["day"] != df["day"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["low"].groupby(df["grouper"]).min().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_low" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def weekly_low(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["week"] != df["week"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["low"].groupby(df["grouper"]).min().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_low" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]
    
def daily_open(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["day"] != df["day"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["open"].groupby(df["grouper"]).first().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_open" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def weekly_open(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["week"] != df["week"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["open"].groupby(df["grouper"]).first().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_open" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]
    
def daily_close(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["day"] != df["day"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["close"].groupby(df["grouper"]).last().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_close" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def weekly_close(dataframe, delay):
    df = dataframe.copy()
    df["rule"] = np.where(df["week"] != df["week"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["close"].groupby(df["grouper"]).last().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_close" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def daily_close_sma(dataframe, delay, period):
    df = dataframe.copy()
    df["rule"] = np.where(df["day"] != df["day"].shift(1),1,0)
    df["grouper"] = df.rule.cumsum()
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["close"].groupby(df["grouper"]).last().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "daily_close" + "_" + str(delay)
    service.columns = [field_name]
    sma_field_name = "daily_close_sma" + "_" + str(delay)
    service[sma_field_name] = service[field_name].rolling(period).mean()
    df1 = pd.concat([df, service], axis = 1)
    df1[sma_field_name] = df1[sma_field_name].fillna(method = "ffill")
    return df1[sma_field_name]

def session_high(dataframe, session_hour, session_minute, delay):
    df = dataframe.copy()
    df["rule"] = np.where((df["hour"] == session_hour) & (df["minute"] == session_minute),1,0)
    df["grouper"] = df.rule.cumsum()
    df = df[df.grouper > 0] #innesto per cancellare i record precedenti al primo trigger
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["high"].groupby(df["grouper"]).max().shift(delay))#.dropna()
    service.set_index(indexes, inplace = True)  
    field_name = "session_high" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def session_low(dataframe, session_hour, session_minute, delay):
    df = dataframe.copy()
    df["rule"] = np.where((df["hour"] == session_hour) & (df["minute"] == session_minute),1,0)    
    df["grouper"] = df.rule.cumsum()
    df = df[df.grouper > 0] #innesto per cancellare i record precedenti al primo trigger
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["low"].groupby(df["grouper"]).min().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "session_low" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def session_open(dataframe, session_hour, session_minute, delay):
    df = dataframe.copy()
    df["rule"] = np.where((df["hour"] == session_hour) & (df["minute"] == session_minute),1,0)  
    df["grouper"] = df.rule.cumsum()
    df = df[df.grouper > 0] #innesto per cancellare i record precedenti al primo trigger
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["open"].groupby(df["grouper"]).first().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "session_open" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]
       
def session_close(dataframe, session_hour, session_minute, delay):
    df = dataframe.copy()
    df["rule"] = np.where((df["hour"] == session_hour) & (df["minute"] == session_minute),1,0) 
    df["grouper"] = df.rule.cumsum()
    df = df[df.grouper > 0] #innesto per cancellare i record precedenti al primo trigger
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["close"].groupby(df["grouper"]).last().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "session_close" + "_" + str(delay)
    service.columns = [field_name]
    df1 = pd.concat([df, service], axis = 1)
    df1[field_name] = df1[field_name].fillna(method = "ffill")
    return df1[field_name]

def session_close_sma(dataframe, session_hour, session_minute, delay, period):
    df = dataframe.copy()
    df["rule"] = np.where((df["hour"] == session_hour) & (df["minute"] == session_minute),1,0) 
    df["grouper"] = df.rule.cumsum()
    df = df[df.grouper > 0] #innesto per cancellare i record precedenti al primo trigger
    indexes = df[df.rule == 1].index
    service = pd.DataFrame(df["close"].groupby(df["grouper"]).last().shift(delay))
    service.set_index(indexes, inplace=True)  
    field_name = "session_close" + "_" + str(delay)
    service.columns = [field_name]
    sma_field_name = "session_close_sma" + "_" + str(delay)
    service[sma_field_name] = service[field_name].rolling(period).mean()
    df1 = pd.concat([df, service], axis = 1)
    df1[sma_field_name] = df1[sma_field_name].fillna(method = "ffill")
    return df1[sma_field_name]

def adx(data,period):
    """
    Function to calculate average directional index for a period of 14 days
    Inputs: dataframe of prices ["open","high","low","close"]
    Output: adx
    """
    data["+DM"] = np.where((data.high - data.high.shift(1) > data.low - data.low.shift(1)) & (data.high - data.high.shift(1)>0),\
                          data.high - data.high.shift(1),0)
    data["-DM"] = np.where((data.high - data.high.shift(1) < data.low - data.low.shift(1)) & (data.low - data.low.shift(1)>0),\
                          data.low - data.low.shift(1),0)
    data["m1"] = data.high - data.low
    data["m2"] = abs(data.high - data.close.shift(1))
    data["m3"] = abs(data.low - data.close.shift(1))
    data["TR"] = data[["m1", "m2", "m3"]].max(axis = 1)
    data["+DM14"] = data["+DM"].rolling(period).sum()
    data["-DM14"] = data["-DM"].rolling(period).sum()
    data["TR14"] = data["TR"].rolling(period).sum()
    data["+DI"] = round(((data["+DM14"]/data["TR14"])*100), 0)
    data["-DI"] = round(((data["-DM14"]/data["TR14"])*100), 0)
    data["DX"] = round(abs((data["+DI"]-data["-DI"])/(data["+DI"]+data["-DI"])),0)
    data["ADX"] = (data.DX.rolling(period).mean())*100
    return data.ADX

def trix(data,period):
    """
    Function to calculate triple exponential moving average
    Inputs: dataframe of prices ["open","high","low","close"]
    Output: trix
    """
    data["MOV_1"] = data.close.ewm(span=period, adjust=False).mean()
    data["MOV_2"] = data.MOV_1.ewm(span=period, adjust=False).mean()
    data["MOV_3"] = data.MOV_2.ewm(span=period, adjust=False).mean()
    data["TRIX"] = ((data.MOV_3 - data.MOV_3.shift(1))/data.MOV_3.shift(1))*100
    return data.TRIX

def Keltner_Channel(df, kc_lookback, multiplier, atr_lookback):
    high=df.high
    low=df.low
    close=df.close
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift()))
    tr3 = pd.DataFrame(abs(low - close.shift()))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(alpha = 1/atr_lookback).mean()
    
    kc_middle = close.ewm(kc_lookback).mean()
    kc_upper = close.ewm(kc_lookback).mean() + multiplier * atr
    kc_lower = close.ewm(kc_lookback).mean() - multiplier * atr
    
    return kc_upper, kc_lower

In [ ]:
from IPython.display import clear_output
import time
import random

np.random.seed(None)

def rnd_gene():
    return np.random.randint(0, rules.shape[0] - 1)


# TODO: create numpy array directly
# build a random dna
def rnd_dna():
    dna = []
    for _ in range(DNA_SIZE):
        dna.append(rnd_gene())
    return np.array(dna)


# build first population at random
def first_generation():
    population = []
    for _ in range(POPULATION_SIZE):
        population.append(rnd_dna())
    return population

def dna_formula(dna):
    formula = ""
    for gene in dna:
        if len(formula) != 0:
            formula += " & "
        formula += "rules["+str(gene)+"]"
    return formula

"""def dna_formula(dna):
    formula = ""
    for gene in dna:
        if len(formula) != 0:
            formula += " & "
        formula += rule_formulas[gene]
    return formula"""

def sort_population(population):
    scores = list(map(fitness, population))
    sorted_population = [dna for _, dna in sorted(zip(scores, population), key=lambda pair: pair[0], reverse=True)]
    return sorted_population

def mutate(dna1):
    dna = dna1.copy()
    dna[np.random.randint(DNA_SIZE)] = rnd_gene()
    return dna


# cross two dnas
def cross(dna1, dna2):
    dna = []
    for i in range(DNA_SIZE):
        if np.random.randint(2) == 0:
            dna.append(dna1[i])
        else:
            dna.append(dna2[i])
    return np.array(dna)



def next_generation(population):

    n = len(population)
    n_best = int(n * BEST_DNA_RATIO)
    n_cross = int(n * CROSS_DNA_RATIO)
    n_mutate = n - n_best - n_cross

    new_population = np.empty_like(population)

    # copy best dnas
    for i in range(n_best):
        new_population[i] = population[i]

    # cross
    for i in range(n_cross):
        #dna1 = population[np.random.randint(n_best)]
        #dna2 = population[np.random.randint(n_best)]
        dna1 = population[np.random.randint(n)]
        dna2 = population[np.random.randint(n)]
        dna = cross(dna1, dna2)
        new_population[n_best + i] = dna

    # mutate
    for i in range(n_mutate):
        #dna1 = population[np.random.randint(n_best)]
        dna1 = population[np.random.randint(n_best)]
        dna = mutate(dna1)
        new_population[n_best + n_cross + i] = dna

    return new_population


def evolution():
    np.random.seed(None)
    best_dnas = []
    population = first_generation()
    population = sort_population(population)
    best_dnas.append(population[0])
    print_best(population, 0)

    # evolve for NUM_GENERATIONS generations
    for i in tqdm(range(1, NUM_GENERATIONS)):
        population = next_generation(population)
        population = sort_population(population)
        best_dnas.append(population[0])
        print_best(population, i)
    
    return best_dnas

def dna_formula_dataframe(dna):
    formula = ""
    for gene in dna:
        if len(formula) != 0:
            formula += " & "
        formula += "dataset_ind.iloc[:,"+str(gene)+"]"
    return formula



def target(direzione,_search,dataset):

    price=dataset.open.copy()
    t_m=[]
    t_m=price.shift(-_search)-price
    t_m=remove_outlier(t_m)
    t_m=t_m.shift(-1)
    t_m=t_m.fillna(0)
    if direzione == "long":
        t_m=np.array(t_m * bigpointvalue)
    else:
        t_m=t_m*(-1)
        t_m=np.array(t_m * bigpointvalue)
    return t_m

def check_level(level,direzione,tipo_ordine):
    
    level=eval(level)
    
    if (direzione == "long") & (tipo_ordine == "stop"):  
        level_check = (history.high >= level)
        
    if (direzione == "short") & (tipo_ordine == "stop"):    
        level_check = (history.low <= level)
        
    if (direzione == "long") & (tipo_ordine == "limit"):  
        level_check = (history.low <= level)
        
    if (direzione == "short") & (tipo_ordine == "limit"):    
        level_check = (history.high >= level)
                            
    return level_check.T.values



def kestner_ratio_mod(operations):

    import numpy as np
    import matplotlib.pyplot as plt 
    from scipy import stats
    
    equity = operations.cumsum()
    index = np.array(np.arange(1,equity.count() + 1))
    
    x = index
    y = equity
    gradient, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    
    if std_err != 0 and len(index) > 0:
        return round(gradient / (std_err * len(index)),5)
    else:
        return np.inf
    
def traduci(x):
    tmp= x.split(" & ")
    stringa=""
    if len(tmp) >1:
        for e1 in range(len(tmp)):
            tmp2=eval(tmp[e1]+'.name')
            tmp2="("+tmp2+")"
            if e1 != (len(tmp)-1):
                stringa=stringa+tmp2+" & "
            else:
                stringa=stringa+tmp2
    else:
        tmp2=eval(tmp[0]+'.name')
        stringa="("+tmp2+")"
        
    return stringa     

"""def fitness(dna0):
    
    tmp=dna_formula(dna0)
    tmp0 = eval(tmp)
    tf0 = strategy(tmp0,exit,target_monetario,costi,level_check,tipo_ordine)
    tf=pd.Series(index=list(range(len(tf0))),data=tf0)
    
    if len(tf)>min_n_operazioni:
        if ranking == "avg_trade":
            gain=avg_trade(tf)
        if ranking == "profit_factor":
            gain=profit_factor(tf)
        if ranking == "profit_factor":
            gain=percent_win(tf)
        if ranking == "profit":
            gain=tf.sum()
        if gain != np.nan:
            return gain
        else:
            return -100
    else:
        return -100
    
# print performance of best dna
def print_best(population, gen_num):
    best_dna = population[0]    
    tmp=dna_formula(best_dna)
    tmp=eval(tmp)
    tf0 = strategy(tmp,exit,target_monetario,costi,level_check,tipo_ordine)
    tf=pd.Series(index=list(range(len(tf0))),data=tf0)
    if len(tf)>1:
        clear_output(wait=True)
        tf.cumsum().plot(figsize=(5,5),label="Generazione n°:"+str(gen_num))
        plt.legend()
        plt.show()
    return tf"""

In [ ]:
from numba import jit
@jit(nopython=True)
def strategy(tmp,exit,target_monetario,costi,level_check,tipo_ordine,ADD_RULE):
    
    tmp = tmp & level_check & ADD_RULE 
    for el in range(len(tmp)):
        if tmp[el] == 1:
            tmp[el+1:el+exit+1]=0
    d=tmp*target_monetario
    d[d != 0 ]=d[d != 0 ]-costi
    return d

@jit(nopython=True)
def avg_trade_numba(array):
    array=array[array != 0]
    avg_t=sum(array)/array.size
    return avg_t

In [ ]:
def performance_report_fast(tradelist,
                       closed_equity,
                       open_equity,
                       initial_capital,
                       risk_free_rate,
                       interactive):
    print("********************************************************************")
    print("*** Performance Report - Copyright 2020 - by Gandalf Project R&D ***")
    print("********************************************************************")
    if tradelist.empty:
        print("")
        print("Nessuna operazione registrata!")
        return
    else: 
        print("")
        print("Compound Annual Growth Rate CAGR: " + str(cagr(tradelist.operations,initial_capital)) + " (capital = " + str(initial_capital) + ")")
        print("Annual Return: " + str(annual_return(tradelist.operations,initial_capital)) + " (capital = " + str(initial_capital) + ")")
        print("")
        print("Calmar Ratio (yearly):", calmar_ratio(tradelist.operations))
        print("Sharpe Ratio: " + str(sharpe_ratio(tradelist.operations, initial_capital, risk_free_rate)) + " (capital = " + str(initial_capital) + ", risk free rate = " + str(risk_free_rate) + ")")
        print("Sortino Ratio: " + str(sortino_ratio(tradelist.operations, initial_capital, risk_free_rate / 12)) + " (capital = " + str(initial_capital) + ", risk free rate = " + str(risk_free_rate / 12) + ")")
        print("Omega Ratio: " + str(omega_ratio(tradelist.operations,100)) + " (threshold = 100)") 
        print("Kestner Ratio:", kestner_ratio(tradelist.operations))
        print("")
        print("Profit:                  ", int(tradelist.operations.sum()))
        print("Operations:              ", operation_number(tradelist.operations))
        print("Average Trade:           ", avg_trade(tradelist.operations))
        print("")
        print("Profit Factor:           ", profit_factor(tradelist.operations))
        print("Gross Profit:            ", gross_profit(tradelist.operations))
        print("Gross Loss:              ", gross_loss(tradelist.operations))
        print("")
        print("Percent Winning Trades:  ", percent_win(tradelist.operations))
        print("Percent Losing Trades:   ", round(100 - percent_win(tradelist.operations),2))
        print("Reward Risk Ratio:       ", reward_risk_ratio(tradelist.operations))
        print("")
        print("Max Gain:                ", max_gain(tradelist.operations), " in date ", max_gain_date(tradelist.operations))
        print("Average Gain:            ", avg_gain(tradelist.operations))
        print("Max Loss:                ", max_loss(tradelist.operations), " in date ", max_loss_date(tradelist.operations))
        print("Average Loss:            ", avg_loss(tradelist.operations))
        print("")
        print("Avg Delay Between Peaks: ", avg_delay_between_peaks(tradelist.operations.cumsum()))
        print("Max Delay Between Peaks: ", max_delay_between_peaks(tradelist.operations.cumsum()))
        print("")
        print("Trades Standard Deviation: ", tradelist.operations.std())
        print("Equity Standard Deviation: ", tradelist.operations.cumsum().std())
        print("")
        print("Avg Open Draw Down:      ", avgdrawdown_nozero(tradelist.operations.cumsum()))
        print("Max Open Draw Down:      ", max_draw_down(tradelist.operations.cumsum()))
        print("")
        print("Avg Closed Draw Down:    ", avgdrawdown_nozero(tradelist.operations.cumsum()))
        print("Max Closed Draw Down:    ", max_draw_down(tradelist.operations.cumsum()))
        print("")
        print("Draw Down Statistics: ", drawdown_statistics(tradelist.operations.cumsum()))
        print("")
        
        print("Operations Statistics:\n")

        if interactive == False:
            plot_equity(tradelist.operations.cumsum(),"green")
            plot_drawdown(tradelist.operations.cumsum(),"red")
            plot_annual_histogram(tradelist.operations)
            plot_monthly_bias_histogram(tradelist.operations)
            plot_equity_heatmap(tradelist.operations,False)

        else:
            plot_equity_interactive(tradelist.operations.cumsum(),"green")
            plot_drawdown_interactive(tradelist.operations.cumsum())
            plot_annual_histogram_interactive(tradelist.operations)
            plot_monthly_bias_histogram_interactive(tradelist.operations)
            plot_equity_heatmap_interactive(tradelist.operations,False)

        return
    
def info_simboli():
    import os
    import numpy as np
    import pandas as pd

    valori=pd.read_csv("Markets_Specifications_Live.csv",sep=",",encoding='cp1252')
    valori.reset_index(inplace=True)
    valori.Exchange_Time=np.where(valori.Exchange_Time==-6,"CST6CDT",valori.Exchange_Time)
    valori.Exchange_Time=np.where(valori.Exchange_Time=="-5","EST5EDT",valori.Exchange_Time)
    valori.Exchange_Time=np.where(valori.Exchange_Time=="1","CET",valori.Exchange_Time)
    valori["Simbolo"]=valori.Physical
    display(valori)

In [ ]:
def level_to_condition(enter_level,history,tipo_ordine,direzione):
    enter_level=eval(enter_level)
    history["level_check"] = [True]*len(enter_level)
    if (tipo_ordine == "stop") | (tipo_ordine == "limit"):
        if (direzione == "long") & (tipo_ordine == "stop"):  
            history["level_check"] = (history.high >= enter_level)
        if (direzione == "short") & (tipo_ordine == "stop"):    
            history["level_check"] = (history.low <= enter_level)
        if (direzione == "long") & (tipo_ordine == "limit"):  
            history["level_check"] = (history.low <= enter_level)
        if (direzione == "short") & (tipo_ordine == "limit"):    
            history["level_check"] = (history.high >= enter_level)

    return history["level_check"]

def check_stoploss(SL,history,enter_level,BIGPOINTVALUE,direzione):

    if (SL != 0) & (direzione == "long") :
        return ((history.close - enter_level) * BIGPOINTVALUE) <= -SL
    if (SL != 0) & (direzione == "short") :
        return ((enter_level - history.close ) * BIGPOINTVALUE) <= -SL
    else:
        return False
    
def check_takeprofit(TP,history,enter_level,BIGPOINTVALUE,direzione):

    if (TP != 0) & (direzione == "long"):
        return ((history.close - enter_level) * BIGPOINTVALUE) >= TP
    if (TP != 0) & (direzione == "short"):
        return ((enter_level - history.close) * BIGPOINTVALUE) >= TP
    else:
        return False    
    
def check_add_rule(ADD_RULE_CONDITION,history):
    if str(ADD_RULE_CONDITION) == "False":
        return np.array([True]*len(history.iloc[:,0]))
    else:
        history["add_rule_check"]=eval(ADD_RULE_CONDITION)
        return np.array(history["add_rule_check"])

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import random

def strategy_numba(idx,bigpointvalue,tick,costi,direzione,exit_bars,level,tipo_ordine,TP,SL,DATAFRAME,ADD_RULE_CONDITION):
    
    history=DATAFRAME.copy()
    
    history["signal"] = eval(idx) 
    
    history["level_to_condition"] = level_to_condition(level,history,tipo_ordine,direzione)
    
    history["add_condition"] = check_add_rule(ADD_RULE_CONDITION,history)
        
    history["level"] = eval(level)
    
    WRITELOG = False

    NAME = "Tester"

    COSTS_FIXED = costi/2
    # 10 for 10 $ for each transaction (2 * fee each operation)
    COSTS_VARIABLE = 0.0
    # 1.0 for 1% (2 * fee each operation)
    COSTS_PERSHARES = 0.0 
    # 0.01 for 1 cent for each shares traded (2 * fee each operation)

    INSTRUMENT = 2 # 1: equity, 2: future, 3: crypto/forex
    TICK = tick
    BIGPOINTVALUE = bigpointvalue

    ORDER_TYPE = "market"

    QUANTITY = 1
    MARGIN_PERCENT = 20 # margin to use in capital calculation

    MAX_INTRADAY_OPERATIONS = 100

    TIME_EXIT = exit_bars  # 0 to disable
    TIME_EXIT_LOSS = 0  # 0 to disable
    TIME_EXIT_GAIN = 0  # 0 to disable

    MONEY_STOPLOSS = 0 # 0 to disable
    MONEY_TARGET = 0  # 0 to disable

    MIN_MONEY_PERCENT_STOPLOSS = 0
    PERCENT_STOPLOSS = 0.0  # 0 to disable

    MIN_MONEY_PERCENT_TARGET = 0
    PERCENT_TARGET = 0.0  # 0 to disable      
    
    enter_level = history["level"]
        
    enter_rules = history["signal"] & history["level_to_condition"] & history["add_condition"]
    
    exit_rules = False 
    
    exit_rules_loss = check_stoploss(SL,history,enter_level,BIGPOINTVALUE,direzione)

    exit_rules_gain = check_takeprofit(TP,history,enter_level,BIGPOINTVALUE,direzione) 

    target_level =  0  # 0 to disable

    stop_level = 0  # 0 to disable

    ##############################################################################################################################################################################################################################
    if direzione == "long":
    
        DIRECTION = "long"

        tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history, INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                                       COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                                       ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                                       exit_rules, exit_rules_loss, exit_rules_gain,
                                                                                       target_level, stop_level,
                                                                                       TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                                       MONEY_STOPLOSS, MONEY_TARGET, 
                                                                                       PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                                       PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
        if len(tradelist)>0:
            return tradelist.operations
        else:
            tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history.iloc[:-1], INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                               COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                               ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                               exit_rules, exit_rules_loss, exit_rules_gain,
                                                                               target_level, stop_level,
                                                                               TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                               MONEY_STOPLOSS, MONEY_TARGET, 
                                                                               PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                               PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
            if len(tradelist)>0:
                return tradelist.operations
            else:
                return pd.DataFrame(columns=["operations"])
    
    ##############################################################################################################################################################################################################################
    
    if direzione == "short":
    
        DIRECTION = "short"

        tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history, INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                                       COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                                       ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                                       exit_rules, exit_rules_loss, exit_rules_gain,
                                                                                       target_level, stop_level,
                                                                                       TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                                       MONEY_STOPLOSS, MONEY_TARGET, 
                                                                                       PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                                       PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
        if len(tradelist)>0:
            return tradelist.operations
        else:
            tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history.iloc[:-1], INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                               COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                               ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                               exit_rules, exit_rules_loss, exit_rules_gain,
                                                                               target_level, stop_level,
                                                                               TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                               MONEY_STOPLOSS, MONEY_TARGET, 
                                                                               PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                               PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
            if len(tradelist)>0:
                return tradelist.operations
            else:
                return pd.DataFrame(columns=["operations"])
        
        

In [ ]:

def strategy_numba_val(v4,bigpointvalue,tick,stampa,v3,tipo_ordine,v2,direzione,v1,v5):
    
    
    WRITELOG = False

    NAME = "Tester"

    COSTS_FIXED = costi/2
    # 10 for 10 $ for each transaction (2 * fee each operation)
    COSTS_VARIABLE = 0.0
    # 1.0 for 1% (2 * fee each operation)
    COSTS_PERSHARES = 0.0 
    # 0.01 for 1 cent for each shares traded (2 * fee each operation)

    INSTRUMENT = 2 # 1: equity, 2: future, 3: crypto/forex
    TICK = tick
    BIGPOINTVALUE = bigpointvalue

    ORDER_TYPE = "market"

    QUANTITY = 1
    MARGIN_PERCENT = 20 # margin to use in capital calculation

    MAX_INTRADAY_OPERATIONS = 100

    TIME_EXIT = v1  # 0 to disable
    TIME_EXIT_LOSS = 0  # 0 to disable
    TIME_EXIT_GAIN = 0  # 0 to disable

    MONEY_STOPLOSS = 0 # 0 to disable
    MONEY_TARGET = 0  # 0 to disable

    MIN_MONEY_PERCENT_STOPLOSS = 0
    PERCENT_STOPLOSS = 0.0  # 0 to disable

    MIN_MONEY_PERCENT_TARGET = 0
    PERCENT_TARGET = 0.0  # 0 to disable

    enter_level = v3

    enter_rules = v4
    
    exit_rules = False 

    exit_rules_loss = check_stoploss(v2,history,enter_level,BIGPOINTVALUE,direzione)

    exit_rules_gain = check_takeprofit(v5,history,enter_level,BIGPOINTVALUE,direzione) 

    target_level =  0  # 0 to disable

    stop_level = 0  # 0 to disable

    ##############################################################################################################################################################################################################################
    if direzione == "long":
    
        DIRECTION = "long"

        tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history, INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                                       COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                                       ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                                       exit_rules, exit_rules_loss, exit_rules_gain,
                                                                                       target_level, stop_level,
                                                                                       TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                                       MONEY_STOPLOSS, MONEY_TARGET, 
                                                                                       PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                                       PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
        if len(tradelist)>0:
            return tradelist.operations
        else:
            tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history.iloc[:-1], INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                               COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                               ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                               exit_rules, exit_rules_loss, exit_rules_gain,
                                                                               target_level, stop_level,
                                                                               TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                               MONEY_STOPLOSS, MONEY_TARGET, 
                                                                               PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                               PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
            if len(tradelist)>0:
                return tradelist.operations
            else:
                return pd.DataFrame(columns=["operations"])
    
    ##############################################################################################################################################################################################################################
    
    if direzione == "short":
    
        DIRECTION = "short"

        tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history, INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                                       COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                                       ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                                       exit_rules, exit_rules_loss, exit_rules_gain,
                                                                                       target_level, stop_level,
                                                                                       TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                                       MONEY_STOPLOSS, MONEY_TARGET, 
                                                                                       PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                                       PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
        if len(tradelist)>0:
            return tradelist.operations
        else:
            tradelist, open_equity, closed_equity, operation_equity = apply_trading_system(history.iloc[:-1], INSTRUMENT, QUANTITY, MARGIN_PERCENT, BIGPOINTVALUE, TICK, DIRECTION,
                                                                               COSTS_FIXED, COSTS_VARIABLE, COSTS_PERSHARES, 
                                                                               ORDER_TYPE, enter_level, enter_rules, MAX_INTRADAY_OPERATIONS, 
                                                                               exit_rules, exit_rules_loss, exit_rules_gain,
                                                                               target_level, stop_level,
                                                                               TIME_EXIT, TIME_EXIT_LOSS, TIME_EXIT_GAIN, 
                                                                               MONEY_STOPLOSS, MONEY_TARGET, 
                                                                               PERCENT_STOPLOSS, MIN_MONEY_PERCENT_STOPLOSS, 
                                                                               PERCENT_TARGET, MIN_MONEY_PERCENT_TARGET, WRITELOG)
            if len(tradelist)>0:
                return tradelist.operations
            else:
                return pd.DataFrame(columns=["operations"])

In [ ]:
def check_history_name(file_name,dir_history):
    import os
    storici = os.listdir(dir_history)
    for e in storici:
        x=e.split("_")[0]
        x=x.replace("@","")
        if x == file_name:
            file_name=e
            break
    return file_name

def last_price_std(valori):
    last_price_list=[]
    std_list=[]
    for e in tqdm(valori.Physical):
        file_name=check_history_name(e,dir_history)
        os.chdir(dir_history)
        if file_name!=e:
            df=pd.read_csv(file_name)
            l_close=df.close.iloc[-1]
            last_price_list.append(l_close)
            std_list.append(df.close.std())
        else:
            last_price_list.append(0)
            std_list.append(0)

    return last_price_list , std_list

In [ ]:
import pandas as pd
import os

def carica_storico_fast(simbolo,uct_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop):
    import os
    import talib as ta
      
    file_name = check_history_name(simbolo,dir_history)

    os.chdir(dir_history)
    data = load_data_intraday_fast(file_name)  
    print("Caricato storico")
    data = data.sort_index(ascending=True)
    data.index = data.index.tz_localize('Etc/Zulu')
    data.index = data.index.tz_convert(uct_offset )
    data.index = data.index.tz_localize(None)
    data = data[data!=0]
    data = data.iloc[1:-1]
    print("Resample dei dati")
    if type_session == 1:
        dataset=resample_custom_session("5min",resample_tf ,data,custom_session_start,custom_session_stop) 
    if type_session == 0:    
        dataset=resample_standard_session("5min",resample_tf,data)
        
    dataset = dataset.loc[IS:OOS]
    dataset["dayofweek"] = dataset.index.dayofweek
    dataset["day"] = dataset.index.day
    dataset["month"] = dataset.index.month
    dataset["year"] = dataset.index.year
    dataset["dayofyear"] = dataset.index.dayofyear
    dataset["quarter"] = dataset.index.quarter
    dataset["hour"] = dataset.index.hour
    dataset["minute"] = dataset.index.minute
    dataset["daily_open"] = daily_open(dataset,1)
    dataset["daily_high"] = daily_high(dataset,1)
    dataset["daily_low"] = daily_low(dataset,1)
    dataset["daily_close"] = daily_close(dataset,1)
    print("Fatto")
    return dataset 

In [ ]:
def backtest_strategy(data,n_id):
    #n_id=4
    costi=data.costi.iloc[n_id]
    direzione=data.direzione.iloc[n_id]
    x=data.entry_rules.iloc[n_id]
    bigpointvalue=data.bigpointvalue.iloc[n_id]
    tick=data.tick.iloc[n_id]
    level=data.level.iloc[n_id]
    tipo_ordine=data.tipo_ordine.iloc[n_id]
    exit_bars=data.exit_bars.iloc[n_id]
    TP=data.TP.iloc[n_id]
    SL=data.SL.iloc[n_id]
    file_name=data.file_name.iloc[n_id]
    uct_offset=data.uct_offset.iloc[n_id]
    type_session=data.type_session.iloc[n_id]
    resample_tf=data.resample_tf.iloc[n_id]
    IS=data.start_date.iloc[n_id]
    OOS=data.end_date.iloc[n_id]
    custom_session_start="0"+str(data.custom_session_start.iloc[n_id])
    custom_session_stop=str(data.custom_session_stop.iloc[n_id])
    history_temp=carica_storico_fast(file_name,uct_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop)
    DATAFRAME_temp=history_temp.copy()
    ADD_RULE_CONDITION=data.ADD_RULE_CONDITION.iloc[n_id]
    backtest_strategy=strategy_numba(x,bigpointvalue,tick,costi,direzione,exit_bars,level,tipo_ordine,TP,SL,DATAFRAME_temp,ADD_RULE_CONDITION)
    return backtest_strategy

In [ ]:
from numba import jit
@jit(nopython=True)
def ohlc_noise_adder_n(O, H, L, C, percentage_noise_addiction):

    new_O = np.zeros(len(O))
    new_H = np.zeros(len(H))
    new_L = np.zeros(len(L))
    new_C = np.zeros(len(C))
    i = 0
    while i < len(O):
        factor = np.random.uniform(-1, 1)
        single_range = H[i] - L[i]
        new_O[i] = round(float(O[i]) + factor * (float(percentage_noise_addiction) / 100) 
                         * abs(float(single_range)), 6)
        new_H[i] = round(float(H[i]) + factor * (float(percentage_noise_addiction) / 100) 
                             * abs(float(single_range)), 6)
        new_L[i] = round(float(L[i]) + factor * (float(percentage_noise_addiction) / 100) 
                             * abs(float(single_range)), 6)
        new_C[i] = round(float(C[i]) + factor * (float(percentage_noise_addiction) / 100) 
                             * abs(float(single_range)), 6)
        app = [new_O[i],new_H[i],new_L[i],new_C[i]]
        app.sort()
        new_O[i] = new_O[i]
        new_H[i] = app[-1]
        new_L[i] = app[0]
        new_C[i] = new_C[i]
        i += 1
    return new_O, new_H, new_L, new_C

def Test_Noise(n_test,original_tradelist,pct_noise,pct_pass_test,stampa):

    ls_noise=[]
    for e in tqdm(range(n_test)):
        b=ohlc_noise_adder_n(history.open.values, history.high.values, history.low.values, history.close.values, pct_noise)
        df=history.copy()
        df.open=b[0]
        df.high=b[1]
        df.low=b[2]
        df.close=b[3]
        trade_list_noise=strategy_numba(dna_check,bigpointvalue,tick,costi,direzione,exit_bars,level,tipo_ordine,take_p,stop_l,df,ADD_RULE_CONDITION)
        ls_noise.append(trade_list_noise)
    df_noise=pd.DataFrame(ls_noise).T
    df_noise=df_noise.fillna(0)

    res_ls=[]
    for e in tqdm(range(len(df_noise.columns))):
        tmp_strat=df_noise.iloc[:,e].to_frame()
        tmp_strat=tmp_strat[tmp_strat!=0]
        res_test=testNoise(original_tradelist.to_frame(),tmp_strat,pct_pass_test,stampa)
        res_ls.append(res_test)
        
    return sum(res_ls)

def check_best_dna(best_dnas):
    tmp_rep=[]
    for elm in range(len(best_dnas)):
        tmp = fitness(best_dnas[elm])
        tmp_rep.append(tmp)
    return tmp_rep.index(max(tmp_rep))

In [ ]:
def testNoise(retestOLD,retestNoise,tol,stampa):
    
    pastest=[]
    fil = ["operation_number","profit","profit_factor","avg_trade","percent_win","reward_risk_ratio","maxdrawdown","avgdrawdown"]
    ts1=pd.DataFrame()
    for e in (retestOLD.columns.tolist()):
        a1=operation_number(retestOLD[e])
        a2=retestOLD[e].sum()
        a3=profit_factor(retestOLD[e])
        a4=avg_trade(retestOLD[e])
        a5=percent_win(retestOLD[e])
        a6=reward_risk_ratio(retestOLD[e])
        a7=max_draw_down(retestOLD[e])
        a8=avgdrawdown_nozero(retestOLD[e])
        a9=[a1,a2,a3,a4,a5,a6,a7,a8]
        a10=pd.DataFrame()
        a10=pd.DataFrame([a9]).T
        a10.columns=([e])
        ts1=pd.concat([ts1,a10],axis=1)
    ts1=ts1.T
    ts1.columns=(fil)

    ts2=pd.DataFrame()
    for e in (retestNoise.columns.tolist()):
        a1=operation_number(retestNoise[e])
        a2=retestNoise[e].sum()
        a3=profit_factor(retestNoise[e])
        a4=avg_trade(retestNoise[e])
        a5=percent_win(retestNoise[e])
        a6=reward_risk_ratio(retestNoise[e])
        a7=max_draw_down(retestNoise[e])
        a8=avgdrawdown_nozero(retestNoise[e])
        a9=[a1,a2,a3,a4,a5,a6,a7,a8]
        a10=pd.DataFrame()
        a10=pd.DataFrame([a9]).T
        a10.columns=([e])
        ts2=pd.concat([ts2,a10],axis=1)
    ts2=ts2.T
    ts2.columns=(fil)

    for e in range (len(ts1)):

        ts1_string = "Original"
        ts2_string = "Noisy"

        ts1_nop = ts1.operation_number[e]
        ts2_nop = ts2.operation_number[e]
        ts1_prft = ts1.profit[e]
        ts2_prft = ts2.profit[e]
        ts1_pf = ts1.profit_factor[e]
        ts2_pf = ts2.profit_factor[e]
        ts1_at = ts1.avg_trade[e]
        ts2_at = ts2.avg_trade[e]
        ts1_pw = ts1.percent_win[e]
        ts2_pw = ts2.percent_win[e]
        ts1_rrr = ts1.reward_risk_ratio[e]
        ts2_rrr = ts2.reward_risk_ratio[e]
        ts1_mdd = ts1.maxdrawdown[e]
        ts2_mdd = ts2.maxdrawdown[e]
        ts1_add = ts1.avgdrawdown[e]
        ts2_add = ts2.avgdrawdown[e]

        count=[]
        if stampa==1:
            print("")
            print("DELTA STATISTICS",ts1.index[e],":")
        
        try:
            if stampa==1:
                print("Operations Number  :", ts1_string, ts1_nop, ts2_string, ts2_nop, 
                      "-> delta:", -round((ts1_nop-ts2_nop)/ts1_nop*100), "%")
            x=round((ts1_nop-ts2_nop)/ts1_nop*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
            if stampa==1:
                print("Profit             :", ts1_string, round(ts1_prft,2), ts2_string, round(ts2_prft,2), 
                      "-> delta:", -round((ts1_prft-ts2_prft)/ts1_prft*100), "%")
            x=round((ts1_prft-ts2_prft)/ts1_prft*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
            if stampa==1:
                print("Profit Factor      :", ts1_string, round(ts1_pf,2), ts2_string, round(ts2_pf,2), 
                      "-> delta:", -round((ts1_pf-ts2_pf)/ts1_pf*100), "%")
            x=round((ts1_pf-ts2_pf)/ts1_pf*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
            if stampa==1:
                print("Average Trade      :", ts1_string, round(ts1_at,2), ts2_string, round(ts2_at,2), 
                      "-> delta:", -round((ts1_at-ts2_at)/ts1_at*100), "%")
            x=round((ts1_at-ts2_at)/ts1_at*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
            if stampa==1:
                print("Percent Win        :", ts1_string, round(ts1_pw,2), "%", ts2_string, round(ts2_pw,2),"%", 
                      "-> delta:", -round((ts1_pw-ts2_pw)/ts1_pw*100), "%")
            x=round((ts1_pw-ts2_pw)/ts1_pw*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
            if stampa==1:
                print("Reward Risk Ratio  :", ts1_string, round(ts1_rrr,2), ts2_string, round(ts2_rrr,2), 
                      "-> delta:", -round((ts1_rrr-ts2_rrr)/ts1_rrr*100), "%")
            x=round((ts1_rrr-ts2_rrr)/ts1_rrr*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)

            if stampa==1:
                print("Max Draw Down      :", ts1_string, round(ts1_mdd,2), ts2_string, round(ts2_mdd,2), 
                      "-> delta:", -round((ts1_mdd-ts2_mdd)/ts1_mdd*100), "%")
            x=round((ts1_mdd-ts2_mdd)/ts1_mdd*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)
                
            if stampa==1:
                print("Avg Draw Down      :", ts1_string, round(ts1_add,2), ts2_string, round(ts2_add,2), 
                      "-> delta:", -round((ts1_add-ts2_add)/ts1_add*100), "%")
            x=round((ts1_add-ts2_add)/ts1_add*100)
            if (x>=-tol) & (x<=tol):
                count.append(1)

            if sum(count) == 8:
                pastest.append(str(ts1.index[e]))
        except OverflowError:
            continue
        except ValueError:
            continue
        if stampa==1:
            print("")
            print("Passano",sum(count),"metriche su 8")
    return sum(count)

In [ ]:
def check_ranking(split_dataframe,ranking,soglia_minima):

    ls_=[]
    for e in split_dataframe:
        #print(eval(ranking)(e))#75
        ls_.append(eval(ranking)(e))
        media=sum(ls_)/len(ls_)
    #print(media)  

    results_avgt=[]
    for E in ls_:
        results_avgt.append((E<(media*1.5)) & (E>(media*0.5)) & (media>soglia_minima))    

    if sum(results_avgt)==len(ls_):
        #print("OK")
        return 1
    else:
        #print("KO")
        return 0
    
def ret_DD(dataframe):
    x=dataframe.sum()/abs(max_draw_down(dataframe.cumsum()))
    return x

def check_days(split_dataframe):
    #print((split_dataframe.index[-1]-split_dataframe.index[0]).days)
    return (split_dataframe.index[-1]-split_dataframe.index[0]).days

def Split_Validation(dataframe,n_split):
    strategy_agg=dataframe.sum(axis=1).resample("D").sum()
    strategy_agg=strategy_agg[strategy_agg!=0].dropna()
    #strategy_agg.cumsum().plot()
    split_dataframe=np.array_split(strategy_agg,n_split)
    """for e in range(len(split_dataframe)-1):
        plt.axvline(x=split_dataframe[e].index[-1])
    plt.show()"""
    check1=check_ranking(split_dataframe,"ret_DD",1.2)
    check2=check_ranking(split_dataframe,"avg_trade",75)
    check3=check_ranking(split_dataframe,"profit_factor",1.25)
    #check4=check_ranking(split_dataframe,"check_days",check_days(strategy_agg)//n_split)
    if check1+check2+check3>=2:
        print("STRATEGIA OK")
        return 1
    else:
        return 0

In [2]:
def pattern_translator():    
    
    

    pattern_dict = {}
    
    pattern_dict['p1'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p1"] = myBodyD1<(0.1*myRangeD1)']
                          
    pattern_dict['p2'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p2"] = myBodyD1<(0.25*myRangeD1)']  
    
    pattern_dict['p3'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p3"]= myBodyD1<(0.5*myRangeD1)']
    
    pattern_dict['p4'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p4"] = myBodyD1<(0.75*myRangeD1)'] 
    
    pattern_dict['p5'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p5"] = myBodyD1>(0.25*myRangeD1)'] 

    pattern_dict['p6'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p6"] = myBodyD1>(0.5*myRangeD1)'] 
    
    pattern_dict['p7'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p7"]= myBodyD1>(0.75*myRangeD1)'] 
    
    pattern_dict['p8'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'myRangeD1 = dataset.daily_high - dataset.daily_low',
                          'dataset["p8"] = myBodyD1>(1*myRangeD1)'] 
    
    pattern_dict['p9'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                          'daily_high5 = daily_high(dataset,5)',
                          'daily_low5 = daily_low(dataset,5)',
                          'myRangeD5 = daily_high5 - daily_low5',
                          'dataset["p9"] = myBodyD1<(0.5*myRangeD5)']
    
    pattern_dict['p10'] = ['myBodyD1 = round(abs(dataset.daily_open - dataset.daily_close),2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high0 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low0 = daily_low(dataset,10)',  
                           'myRangeMaxD10 = (highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high0) -\
                           (lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low0)))',
                           'dataset["p10"] = myBodyD1<(0.5*myRangeMaxD10)']
    
    pattern_dict['p11'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p11"] = myBodyD5 < (0.1*(daily_high5 - dataset.daily_low))']
    
    pattern_dict['p12'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p12"] = myBodyD5 < (0.25*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p13'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p13"] = myBodyD5 < (0.50*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p14'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p14"] = myBodyD5 < (0.75*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p15'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p15"] = myBodyD5 < (1*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p16'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p16"] = myBodyD5 < (1.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p17'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p17"] = myBodyD5 < (2*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p18'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p18"] = myBodyD5 > (0.25*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p19'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p19"] = myBodyD5 > (0.50*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p20'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p20"] = myBodyD5 > (0.75*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p21'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p21"] = myBodyD5 > (1*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p22'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p22"] = myBodyD5 > (1.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p23'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p23"] = myBodyD5 > (2*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p24'] = ['daily_high5 = daily_high(dataset,5)',
                          'daily_open5 = daily_open(dataset,5)',
                          'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                          'dataset["p24"] = myBodyD5 > (2.5*(daily_high5-dataset.daily_low))']
    
    pattern_dict['p25'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p25"] = myBodyD5 < (0.1*myRangeD5)']
    
    pattern_dict['p26'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p26"] = myBodyD5 < (0.25*myRangeD5)']
    
    pattern_dict['p27'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p27"] = myBodyD5 < (0.5*myRangeD5)']
    
    pattern_dict['p28'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p28"] = myBodyD5 < (0.75*myRangeD5)']
    
    pattern_dict['p29'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p29"] = myBodyD5 > (0.1*myRangeD5)']
    
    pattern_dict['p30'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p30"] = myBodyD5 > (0.25*myRangeD5)']
    
    pattern_dict['p31'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p31"] = myBodyD5 > (0.5*myRangeD5)']
    
    pattern_dict['p32'] = ['daily_high5 = daily_high(dataset,5)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_open5 = daily_open(dataset,5)',
                           'myBodyD5 = round(abs(daily_open5 - dataset.daily_close),2)',
                           'myRangeD5 = daily_high5 - daily_low5',                           
                           'dataset["p32"] = myBodyD5 > (0.75*myRangeD5)']
    
    pattern_dict['p33'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'dataset["p33"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3) & (daily_close3>daily_close4)']
    
    pattern_dict['p34'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'dataset["p34"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3) & (daily_close3<daily_close4)']
    
    pattern_dict['p35'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'daily_close5 = daily_close(dataset,5)',
                           'dataset["p35"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3) & (daily_close3>daily_close4) & (daily_close4>daily_close5)']
    
    pattern_dict['p36'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'daily_close4 = daily_close(dataset,4)',
                           'daily_close5 = daily_close(dataset,5)',
                           'dataset["p35"] = dataset["p36"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3) & (daily_close3<daily_close4) & (daily_close4<daily_close5)']
    
    pattern_dict['p37'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',  
                           'dataset["p37"] = (dataset.daily_high>daily_high2) & (dataset.daily_low>daily_low2)']
    
    pattern_dict['p38'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',  
                           'dataset["p38"] = (dataset.daily_high<daily_high2) & (dataset.daily_low<daily_low2)']
    
    pattern_dict['p39'] = ['daily_close2 = daily_close(dataset,2)',                             
                           'dataset["p39"] = (dataset.daily_close>daily_close2)']
    
    pattern_dict['p40'] = ['daily_close2 = daily_close(dataset,2)',                             
                           'dataset["p40"] = (dataset.daily_close<daily_close2)']
    
    pattern_dict['p41'] = ['dataset["p41"] = (dataset.daily_close<dataset.daily_open)']
    
    pattern_dict['p42'] = ['dataset["p42"] = (dataset.daily_close>dataset.daily_open)']
    
    pattern_dict['p43'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p43"] = (dataset.daily_close<(daily_close2-(daily_close2*0.5*0.01)))']
    
    pattern_dict['p44'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p44"] = (dataset.daily_close<(daily_close2-(daily_close2*1*0.01)))']
    
    pattern_dict['p45'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p45"] = (dataset.daily_close<(daily_close2-(daily_close2* 1.5*0.01)))']
    
    pattern_dict['p46'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p46"] = (dataset.daily_close<(daily_close2-(daily_close2*2*0.01)))']
    
    pattern_dict['p47'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p47"] = (dataset.daily_close<(daily_close2-(daily_close2*2.5*0.01)))']
    
    pattern_dict['p48'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p48"] = (dataset.daily_close<(daily_close2-(daily_close2*3*0.01)))']
    
    pattern_dict['p49'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p49"] = (dataset.daily_close>(daily_close2+(daily_close2*0.5*0.01)))']
    
    pattern_dict['p50'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p50"] = (dataset.daily_close>(daily_close2+(daily_close2*1*0.01)))']
    
    pattern_dict['p51'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p51"] = (dataset.daily_close>(daily_close2+(daily_close2*1.5*0.01)))']
    
    pattern_dict['p52'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p52"] = (dataset.daily_close>(daily_close2+(daily_close2*2*0.01)))']
    
    pattern_dict['p53'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p53"] = (dataset.daily_close>(daily_close2+(daily_close2*2.5*0.01)))']
    
    pattern_dict['p54'] = ['daily_close2 = daily_close(dataset,2)', 
                           'dataset["p54"] = (dataset.daily_close>(daily_close2+(daily_close2*3*0.01)))']
    
    pattern_dict['p55'] = ['daily_high5 = daily_high(dataset,5)', 
                           'dataset["p55"] = dataset.daily_high>daily_high5']
    
    pattern_dict['p56'] = ['daily_high5 = daily_high(dataset,5)', 
                           'dataset["p56"] = dataset.daily_high<daily_high5']
    
    pattern_dict['p57'] = ['daily_low5 = daily_low(dataset,5)', 
                           'dataset["p57"] = dataset.daily_low>daily_low5']
    
    pattern_dict['p58'] = ['daily_low5 = daily_low(dataset,5)', 
                           'dataset["p58"] = dataset.daily_low<daily_low5']
    
    pattern_dict['p59'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p59"] = (dataset.daily_high>daily_high2)&(dataset.daily_high>daily_high3)&(dataset.daily_high>daily_high4)']
    
    pattern_dict['p59'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p59"] = (dataset.daily_high>daily_high2)&(dataset.daily_high>daily_high3)&(dataset.daily_high>daily_high4)']
    
    pattern_dict['p60'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',                           
                           'dataset["p60"] = (dataset.daily_high<daily_high2)&(dataset.daily_high<daily_high3)&(dataset.daily_high<daily_high4)']
    
    pattern_dict['p61'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',                           
                           'dataset["p61"] = (dataset.daily_low<daily_low2)&(dataset.daily_low<daily_low3)&(dataset.daily_low<daily_low4)']
    
    pattern_dict['p62'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',                           
                           'dataset["p62"] = (dataset.daily_low>daily_low2)&(dataset.daily_low>daily_low3)&(dataset.daily_low>daily_low4)']
    
    pattern_dict['p63'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',                                                
                           'dataset["p63"] = (dataset.daily_close>daily_close2)&(dataset.daily_close>daily_close3)&(dataset.daily_close>dataset.daily_open)']
    
    pattern_dict['p64'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',                                                
                           'dataset["p64"] = (dataset.daily_close<daily_close2)&(dataset.daily_close<daily_close3)&(dataset.daily_close<dataset.daily_open)']
    
    pattern_dict['p65'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',                                              
                           'dataset["p65"] = (dataset.daily_high-dataset.daily_close)<0.20*myRangeD1']
    
    pattern_dict['p66'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',                                              
                           'dataset["p66"] = (dataset.daily_high-dataset.daily_low)<0.20*myRangeD1']
    
    pattern_dict['p67'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'dataset["p67"] = (myRangeD1<(((daily_high2-daily_low2)+(daily_high2-daily_low2))/3))']
    
    pattern_dict['p68'] = ['myRangeD1 = dataset.daily_high - dataset.daily_low',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'myRangeD2 = daily_high2 - daily_low2',
                           'myRangeD3 = daily_high3 - daily_low3',
                           'dataset["p68"] = (myRangeD1<myRangeD2)&(myRangeD2<myRangeD3)']
    
    pattern_dict['p69'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'dataset["p69"] = (daily_high2>dataset.daily_high)&(daily_low2 < dataset.daily_low)']
    
    pattern_dict['p70'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p70"] = (dataset.daily_high<daily_high2)']
    
    pattern_dict['p71'] = ['daily_low2 = daily_low(dataset,2)',
                           'dataset["p71"] = (dataset.daily_low>daily_low2)']
    
    pattern_dict['p72'] = ['daily_low2 = daily_low(dataset,2)',
                           'dataset["p72"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p73'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                            
                           'dataset["p73"] = (daily_high2>dataset.daily_high)|(daily_low2<dataset.daily_low)']
    
    pattern_dict['p74'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                            
                           'dataset["p74"] = (daily_high2<dataset.daily_high)&(daily_low2>dataset.daily_low)']
    
    pattern_dict['p75'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p75"] = (dataset.daily_close>dataset.daily_open) & (daily_close2>daily_open2)']
    
    pattern_dict['p76'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p76"] = (dataset.daily_close<dataset.daily_open) & (daily_close2>daily_open2)']
    
    pattern_dict['p77'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',                            
                           'dataset["p77"] = (dataset.daily_close<dataset.daily_open) & (daily_close2<daily_open2)']
    
    pattern_dict['p78'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',                           
                           'myMaxHighD2 = highest_serie(dataset.daily_high,daily_high2)',
                           'myMinLowD2 = lowest_serie(dataset.daily_low,daily_low2)',  
                           'dataset["p78"] = abs(daily_open2-dataset.daily_close) < (0.5*(myMaxHighD2 - myMinLowD2))']
    
    pattern_dict['p79'] = ['daily_high2 = daily_high(dataset,2)',                          
                           'dataset["p79"] = (dataset.daily_high> daily_high2)']
    
    pattern_dict['p80'] = ['daily_high2 = daily_high(dataset,2)',   
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p80"] = (dataset.daily_high> daily_high2) & (dataset.daily_high>daily_high3)']
    
    pattern_dict['p81'] = ['daily_high2 = daily_high(dataset,2)',   
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p81"] = (dataset.daily_high< daily_high2) & (dataset.daily_high< daily_high3)']
    
    pattern_dict['p82'] = ['daily_low2 = daily_low(dataset,2)',                          
                           'dataset["p82"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p82'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)',                            
                           'dataset["p82"] = (dataset.daily_low>daily_low2) & (dataset.daily_low>daily_low3)']
    
    pattern_dict['p83'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)',                            
                           'dataset["p83"] = (dataset.daily_low<daily_low2) & (dataset.daily_low<daily_low3)']
    
    pattern_dict['p84'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)', 
                           'daily_high2 = daily_high(dataset,2)', 
                           'daily_high3 = daily_high(dataset,3)', 
                           'dataset["p84"] = (dataset.daily_high>daily_high2) & (dataset.daily_low>daily_low2) & (daily_high2<daily_high3) & (daily_low2<daily_low3)']
    
    pattern_dict['p85'] = ['daily_low2 = daily_low(dataset,2)', 
                           'daily_low3 = daily_low(dataset,2)', 
                           'daily_high2 = daily_high(dataset,2)', 
                           'daily_high3 = daily_high(dataset,3)', 
                           'dataset["p85"] = (dataset.daily_high<daily_high2) & (dataset.daily_low<daily_low2) & (daily_high2>daily_high3) & (daily_low2>daily_low3)']


    pattern_dict['p86'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'dataset["p86"] = (dataset.daily_close>daily_close2) & (daily_close2>daily_close3)']
    
    pattern_dict['p87'] = ['daily_close2 = daily_close(dataset,2)',
                           'daily_close3 = daily_close(dataset,3)',
                           'dataset["p87"] = (dataset.daily_close<daily_close2) & (daily_close2<daily_close3)']
    
    pattern_dict['p88'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,2)',                           
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high +dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'dataset["p88"] = (myAvgPriceD1 > myAvgPriceD2)']
    
    pattern_dict['p89'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,3)', 
                           'daily_open3 = daily_open(dataset,3)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_close3 = daily_close(dataset,3)', 
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'myAvgPriceD3 = round(((daily_open3 + daily_high3 + daily_low3 + daily_close3)/4),2)',
                           'dataset["p89"] = (myAvgPriceD1 > myAvgPriceD2) & (myAvgPriceD1 > myAvgPriceD3)']
    
    pattern_dict['p90'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,2)',                           
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'dataset["p90"] = (myAvgPriceD1 < myAvgPriceD2)']
    
    pattern_dict['p91'] = ['daily_open2 = daily_open(dataset,2)',
                           'daily_high2 = daily_high(dataset,2)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_close2 = daily_close(dataset,3)', 
                           'daily_open3 = daily_open(dataset,3)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_close3 = daily_close(dataset,3)', 
                           'myAvgPriceD1 = round(((dataset.daily_open + dataset.daily_high + dataset.daily_low + dataset.daily_close)/4),2)',
                           'myAvgPriceD2 = round(((daily_open2 + daily_high2 + daily_low2 + daily_close2)/4),2)',
                           'myAvgPriceD3 = round(((daily_open3 + daily_high3 + daily_low3 + daily_close3)/4),2)',
                           'dataset["p91"] = (myAvgPriceD1 < myAvgPriceD2) & (myAvgPriceD1 < myAvgPriceD3)']

    pattern_dict['p92'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p92"] = (dataset.daily_close>daily_high2)']
    
    pattern_dict['p93'] = ['daily_high2 = daily_high(dataset,2)',
                           'dataset["p93"] = (dataset.daily_close<daily_high2)']
    
    pattern_dict['p94'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'dataset["p94"] = (dataset.daily_close>daily_high2) & (dataset.daily_close>daily_high3)']
    
    pattern_dict['p95'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'dataset["p95"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<daily_low3) ']
    
    pattern_dict['p96'] = ['dataset["p95"] = True']
    
    pattern_dict['p97'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'dataset["p97"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<daily_low3) & (dataset.daily_close<daily_low4)']
    
    pattern_dict['p98'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',
                           'dataset["p98"] = (dataset.daily_close>daily_high2) & (dataset.daily_close>dataset.daily_open) & (dataset.daily_close<daily_open2)']
    
    pattern_dict['p99'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_open2 = daily_open(dataset,2)',
                           'dataset["p99"] = (dataset.daily_close<daily_low2) & (dataset.daily_close<dataset.daily_open) & (dataset.daily_close>daily_open2)']
    
    pattern_dict['p100'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',
                            'myRangeD2 = daily_high2 - daily_low2',
                            'dataset["p100"] = (myRangeD1<myRangeD2)']
    
    pattern_dict['p101'] = ['daily_low5 = daily_low(dataset,5)',
                            'daily_high5 = daily_high(dataset,5)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',
                            'myRangeD5 = daily_high5 - daily_low5',
                            'dataset["p101"] = (myRangeD1<myRangeD5)']
    
    pattern_dict['p102'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high10 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low10 = daily_low(dataset,10)',  
                           'myRangeMaxD10 = (highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high10) - (lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low10)))',
                           'myRangeD1 = dataset.daily_high - dataset.daily_low',                            
                           'dataset["p102"] = (myRangeD1<myRangeMaxD10)']
    
    pattern_dict['p103'] = ['daily_high2 = daily_high(dataset,2)',
                           'daily_high3 = daily_high(dataset,3)',
                           'daily_high4 = daily_high(dataset,4)',
                           'daily_high5 = daily_high(dataset,5)',
                           'daily_high6 = daily_high(dataset,6)',
                           'daily_high7 = daily_high(dataset,7)',
                           'daily_high8 = daily_high(dataset,8)',
                           'daily_high9 = daily_high(dataset,9)',
                           'daily_high0 = daily_high(dataset,10)',
                           'daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',
                           'daily_low6 = daily_low(dataset,5)',
                           'daily_low7 = daily_low(dataset,7)',
                           'daily_low8 = daily_low(dataset,8)',
                           'daily_low9 = daily_low(dataset,9)',
                           'daily_low0 = daily_low(dataset,10)', 
                           'myRangeDelay1MaxD10 = highest_serie(daily_high2,daily_high3,daily_high4,daily_high5,daily_high6,daily_high7,daily_high8,daily_high9,daily_high0) - \
                            lowest_serie(daily_low2,daily_low3,daily_low4,daily_low5,daily_low6,daily_low7,daily_low8,daily_low9,daily_low0)',
                            'myRangeD1 = dataset.daily_high - dataset.daily_low',                            
                            'dataset["p103"] = (myRangeD1>myRangeDelay1MaxD10)']
    

    pattern_dict['p104'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'myMaxDelay1HighD4 =  highest_serie(daily_high2,daily_high3,daily_high4)',
                            'dataset["p104"] = (dataset.daily_close>myMaxDelay1HighD4)']
    
    pattern_dict['p105'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'myMaxDelay1HighD5 = highest_serie(daily_high2,daily_high3,daily_high4,daily_high5)',
                            'dataset["p105"] = (dataset.daily_close>myMaxDelay1HighD5)']
    
    pattern_dict['p106'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                            'myMinDelay1LowD4 = lowest_serie(daily_low2,daily_low3,daily_low4)',
                            'dataset["p106"] = (dataset.daily_close<myMinDelay1LowD4)']
    
    pattern_dict['p107'] = ['daily_low2 = daily_low(dataset,2)',
                           'daily_low3 = daily_low(dataset,3)',
                           'daily_low4 = daily_low(dataset,4)',
                           'daily_low5 = daily_low(dataset,5)',                           
                           'myMinDelay1LowD5 = lowest_serie(daily_low2,daily_low3,daily_low4,daily_low5)',
                           'dataset["p107"] = (dataset.daily_close<myMinDelay1LowD5)']
    
    pattern_dict['p108'] = ['dataset["p108"] = True ']
    pattern_dict['p109'] = ['dataset["p109"] = True ']
    pattern_dict['p110'] = ['dataset["p110"] = True ']
    pattern_dict['p111'] = ['dataset["p111"] = True ']
    pattern_dict['p112'] = ['dataset["p112"] = True ']
    pattern_dict['p113'] = ['dataset["p113"] = True ']

    
    pattern_dict['p114'] = ['daily_close2 = daily_close(dataset,2)',
                            'dataset["p114"] = (dataset.daily_open<daily_close2)']
    
    pattern_dict['p115'] = ['daily_low3 = daily_low(dataset,3)',
                            'dataset["p115"] = (dataset.daily_low<daily_low3)']
    
    pattern_dict['p116'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'dataset["p116"] = (dataset.daily_high-dataset.daily_low)<(daily_high2-daily_low2)']
    
    pattern_dict['p117'] = ['daily_high3 = daily_high(dataset,3)',
                            'dataset["p117"] = (dataset.daily_high>daily_high3)']
    
    pattern_dict['p118'] = ['daily_low2 = daily_low(dataset,2)',
                            'dataset["p118"] = (dataset.daily_low<daily_low2)']
    
    pattern_dict['p119'] = ['daily_high2 = daily_high(dataset,2)',
                            'dataset["p119"] = (dataset.daily_high>daily_high2)']
    
    pattern_dict['p120'] = ['daily_open5 = daily_open(dataset,5)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',
                            'daily_low5 = daily_low(dataset,5)',                            
                            'myMaxHighD5 = highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5)',
                            'myMinLowD5 = lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5)',
                            'dataset["p120"] = (abs(daily_open5-dataset.daily_close)<0.66*(myMaxHighD5-myMinLowD5))']
    
    pattern_dict['p121'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_high5 = daily_high(dataset,5)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',
                            'daily_low5 = daily_low(dataset,5)',                            
                            'myMaxHighD5 = highest_serie(dataset.daily_high,daily_high2,daily_high3,daily_high4,daily_high5)',
                            'myMinLowD5 = lowest_serie(dataset.daily_low,daily_low2,daily_low3,daily_low4,daily_low5)',
                            'dataset["p121"] = (abs(dataset.daily_open-dataset.daily_close)<0.66*(myMaxHighD5-myMinLowD5))']
    
    
    pattern_dict['p122'] = ['dataset["p122"] = (abs(dataset.daily_open-dataset.daily_close)<0.66*(dataset.daily_high-dataset.daily_low))']
        
    pattern_dict['p123'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low4 = daily_low(dataset,4)',                            
                            'dataset["p123"] = (dataset.daily_high-dataset.daily_low)>0.66*(((dataset.daily_high-dataset.daily_low)+\
                            (daily_high2-daily_low2)+(daily_high3-daily_low3)+\
                            (daily_high4-daily_low4)/4))']
    
    pattern_dict['p124'] = ['daily_low2 = daily_low(dataset,2)',
                            'daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low4 = daily_low(dataset,4)',                            
                            'dataset["p124"] = (dataset.daily_high-dataset.daily_low)<0.66*(((dataset.daily_high-dataset.daily_low)+\
                            (daily_high2-daily_low2)+(daily_high3-daily_low3)+\
                            (daily_high4-daily_low4)/4))']
        
    
    pattern_dict['p125'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p125"] = abs((weekly_open1-weekly_close1)<0.75*(weekly_high1-weekly_low1))']
    
    pattern_dict['p125'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p125"] = abs((weekly_open1-weekly_close1)<0.75*(weekly_high1-weekly_low1))']
        
    pattern_dict['p126'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p126"] = abs((weekly_open1-weekly_close1)<0.50*(weekly_high1-weekly_low1))']
    
    pattern_dict['p127'] = ['weekly_low1 = weekly_low(dataset,1)',
                            'weekly_open1 = weekly_open(dataset,1)',
                            'weekly_close1 = weekly_close(dataset,1)',
                            'weekly_high1 = weekly_high(dataset,1)',        
                            'dataset["p127"] = abs((weekly_open1-weekly_close1)<0.25*(weekly_high1-weekly_low1))']
    
    pattern_dict['p128'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p128"] = ((daily_high4-daily_low4)<(daily_high3-daily_low3)) & ((daily_high3-daily_low3)<(daily_high2-daily_low2)) & ((daily_high2-daily_low2)<(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p129'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p129"] = ((daily_high3-daily_low3)<(daily_high2-daily_low2))&((daily_high2-daily_low2)<(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p130'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p130"] = ((daily_high4-daily_low4)>(daily_high3-daily_low3)) & ((daily_high3-daily_low3)>(daily_high2-daily_low2)) & ((daily_high2-daily_low2)>(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p131'] = ['daily_high2 = daily_high(dataset,2)',
                            'daily_high3 = daily_high(dataset,3)',
                            'daily_high4 = daily_high(dataset,4)',
                            'daily_low2 = daily_low(dataset,2)',
                            'daily_low3 = daily_low(dataset,3)',
                            'daily_low4 = daily_low(dataset,4)',       
                            'dataset["p131"] = ((daily_high3-daily_low3)>(daily_high2-daily_low2)) & ((daily_high2-daily_low2)>(dataset.daily_high-dataset.daily_low))']
    
    pattern_dict['p132'] = ['dataset["p132"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p133'] = ['dataset["p133"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p134'] = ['dataset["p134"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/3']
    
    pattern_dict['p135'] = ['dataset["p135"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/4']
    
    pattern_dict['p136'] = ['dataset["p136"] = (dataset.daily_close-dataset.daily_low)<(dataset.daily_high-dataset.daily_low)/2']
    
    pattern_dict['p137'] = ['dataset["p137"] = (dataset.daily_high-dataset.daily_close)<(dataset.daily_high-dataset.daily_low)/2']
    
    pattern_dict['p138'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p138"] = (daily_open0 > dataset.daily_high)']
    
    pattern_dict['p139'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p139"] = (daily_open0<dataset.daily_high)']
    
    pattern_dict['p140'] = ['daily_open0 = daily_open(dataset,0)',                            
                            'dataset["p140"] = (daily_open0<dataset.daily_low)']
    
    pattern_dict['p141'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_high2 = daily_high(dataset,2)',                            
                            'dataset["p141"] = (daily_open0<=daily_high2)']
    
    pattern_dict['p142'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_high2 = daily_high(dataset,2)',                            
                            'dataset["p142"] = (daily_open0>daily_high2)']
    
    pattern_dict['p143'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_low3 = daily_low(dataset,3)',                            
                            'dataset["p143"] = (daily_open0>daily_low3)']
    
    pattern_dict['p144'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_open2 = daily_open(dataset,2)',                            
                            'dataset["p144"] = (daily_open0>daily_open2)']
    
    pattern_dict['p145'] = ['daily_open0 = daily_open(dataset,0)',
                            'daily_open2 = daily_open(dataset,2)',                            
                            'dataset["p145"] = (daily_open0<daily_open2)']
    
    pattern_dict['p146'] = ['daily_low2 = daily_low(dataset,2)',                            
                            'dataset["p146"] = (dataset.daily_close>daily_low2)']
    
    pattern_dict['p147'] = ['daily_low2 = daily_low(dataset,2)',                            
                            'dataset["p147"] = (dataset.daily_close<daily_low2)']
    
    pattern_dict['p148'] = ['daily_close2 = daily_close(dataset,2)',                            
                            'dataset["p148"] = (dataset.daily_close>daily_close2)']
    
    pattern_dict['p149'] = ['daily_close2 = daily_close(dataset,2)',                            
                            'dataset["p149"] = (dataset.daily_close<daily_close2)']
    
    pattern_dict['p150'] = ['daily_close2 = daily_close(dataset,2)',
                            'daily_close3 = daily_close(dataset,3)',                            
                            'dataset["p150"] = (daily_close2 < daily_close3)']

    return pattern_dict


In [ ]:
def carica_storico_second(file_name,second_history,uct_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,noise,pct_noise):
    import os
    import talib as ta
    
    os.chdir(dir_history)
    data = load_data_intraday_fast(file_name)  
    data_s = load_data_intraday_fast(second_history) 
    
    data=data.resample("5min").mean()
    data_s=data_s.resample("5min").mean()
    
    
    data=pd.concat([data,data_s],axis=1).fillna(method="ffill")
    data=data.dropna()
    data=data.loc[IS:]
    data_s=data.iloc[:,5:]
    data=data.iloc[:,:5]
    
    display(data.tail(1),data_s.tail(1))
    print(len(data),len(data_s))
    
    print("Caricato storico")
    data = data.sort_index(ascending=True)
    data.index = data.index.tz_localize('Etc/Zulu')
    data.index = data.index.tz_convert(uct_offset )
    data.index = data.index.tz_localize(None)
    data = data[data!=0]
    data = data.iloc[1:-1]
    
    data_s = data_s.sort_index(ascending=True)
    data_s.index = data_s.index.tz_localize('Etc/Zulu')
    data_s.index = data_s.index.tz_convert(uct_offset )
    data_s.index = data_s.index.tz_localize(None)
    data_s = data_s[data_s!=0]
    data_s = data_s.iloc[1:-1]
    
    print("Resample dei dati")
    if type_session == 1:
        dataset=resample_custom_session("5min",resample_tf ,data,custom_session_start,custom_session_stop)
        dataset_s=resample_custom_session("5min",resample_tf ,data_s,custom_session_start,custom_session_stop)
    if type_session == 0:    
        dataset=resample_standard_session("5min",resample_tf,data)
        dataset_s=resample_standard_session("5min",resample_tf,data_s)
        
    dataset=pd.concat([dataset,dataset_s],axis=1).fillna(method="ffill")
    dataset_s=dataset.iloc[:,5:]
    dataset=dataset.iloc[:,:5]
        
    if noise == 1:
        print("AGGIUNGO RUMORE")
        dataset=add_noise(dataset.open, dataset.high, dataset.low, dataset.close,dataset.volume, pct_noise)

    dataset = dataset.loc[IS:OOS]
    dataset_s = dataset_s.loc[IS:OOS]
    dataset["dayofweek"] = dataset.index.dayofweek
    dataset["day"] = dataset.index.day
    dataset["month"] = dataset.index.month
    dataset["year"] = dataset.index.year
    dataset["dayofyear"] = dataset.index.dayofyear
    dataset["quarter"] = dataset.index.quarter
    dataset["hour"] = dataset.index.hour
    dataset["minute"] = dataset.index.minute
    dataset["daily_open"] = daily_open(dataset,1)
    dataset["daily_high"] = daily_high(dataset,1)
    dataset["daily_low"] = daily_low(dataset,1)
    dataset["daily_close"] = daily_close(dataset,1)
    dataset["AVGPRICE"]=ta.AVGPRICE(dataset.open, dataset.high, dataset.low, dataset.close)
    dataset["MEDPRICE"]=ta.MEDPRICE(dataset.high, dataset.low)
    dataset["TYPPRICE"]=ta.TYPPRICE(dataset.high, dataset.low, dataset.close)
    dataset["WCLPRICE"]=ta.WCLPRICE(dataset.high, dataset.low, dataset.close)
    history=dataset.copy()
    
    print("Carico indicatori")
    import talib as ta
    applica_indicatori=apply_indicator(dataset_s)
    dataset=applica_indicatori[0]
    dataset_ind = dataset.iloc[:,5:].copy()


    rules=dataset.iloc[:,5:].T.values
    rule_formulas=np.arange(len(rules))
    
    history.tail(10)
    
    history.close.plot(figsize=(20,10),title=SIMBOLO+"_"+RESAMPLE_TF)
    print("Fatto")
    return dataset , dataset_ind , history , rules, rule_formulas

def target(direzione,_search,dataset):
    _search=_search
    close=history.close.copy()
    t_m=[]
    if direzione == "long":
        t_m=close-close.shift(_search)
        t_m=t_m.shift(-_search).fillna(0)
        t_m=np.array(t_m * bigpointvalue)
    else:
        t_m=close-close.shift(_search)
        t_m=t_m.shift(-_search).fillna(0)
        t_m=t_m*(-1)
        t_m=np.array(t_m * bigpointvalue)
    return t_m

def environment_second(simbolo,simbolo2,type_session,resample_tf,start_date,end_date):
               
    file_name = check_history_name(simbolo,dir_history)
    file_name2 = check_history_name(simbolo,dir_history) #simbolo2+".csv"

    IS = start_date
    OOS = end_date

    bigpointvalue=valori[valori.Physical==simbolo].BigPointValue.values[0]
    tick=valori[valori.Physical==simbolo].Tick.values[0]
    utc_offset = valori[valori.Physical==simbolo].Exchange_Time.values[0]
    custom_session_start = "0"+str(valori[valori.Physical==simbolo]["Custom Open"].values[0])
    custom_session_stop = str(valori[valori.Physical==simbolo]["Custom Close"].values[0])
    costi=valori[valori.Physical==simbolo].Cost.values[0]

    dataset , dataset_ind , history , rules, rule_formulas = carica_storico_second(file_name,file_name2,utc_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,0,0)
    
    return dataset , dataset_ind , history , rules, rule_formulas , bigpointvalue , tick , utc_offset , custom_session_start , custom_session_stop , costi 



In [ ]:
def carica_storico_second(file_name,second_history,uct_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,noise,pct_noise):
    import os
    import talib as ta
    
    os.chdir(dir_history)
    data = load_data_intraday_fast(file_name)  
    data_s = load_data_intraday_fast(second_history) 
    
    data=data.resample("5min").mean()
    data_s=data_s.resample("5min").mean()
    
    
    data=pd.concat([data,data_s],axis=1).fillna(method="ffill")
    data=data.dropna()
    data=data.loc[IS:]
    data_s=data.iloc[:,5:]
    data=data.iloc[:,:5]
    
    display(data.tail(1),data_s.tail(1))
    print(len(data),len(data_s))
    
    print("Caricato storico")
    data = data.sort_index(ascending=True)
    data.index = data.index.tz_localize('Etc/Zulu')
    data.index = data.index.tz_convert(uct_offset )
    data.index = data.index.tz_localize(None)
    data = data[data!=0]
    data = data.iloc[1:-1]
    
    data_s = data_s.sort_index(ascending=True)
    data_s.index = data_s.index.tz_localize('Etc/Zulu')
    data_s.index = data_s.index.tz_convert(uct_offset )
    data_s.index = data_s.index.tz_localize(None)
    data_s = data_s[data_s!=0]
    data_s = data_s.iloc[1:-1]
    
    print("Resample dei dati")
    if type_session == 1:
        dataset=resample_custom_session("5min",resample_tf ,data,custom_session_start,custom_session_stop)
        dataset_s=resample_custom_session("5min",resample_tf ,data_s,custom_session_start,custom_session_stop)
    if type_session == 0:    
        dataset=resample_standard_session("5min",resample_tf,data)
        dataset_s=resample_standard_session("5min",resample_tf,data_s)
        
    dataset=pd.concat([dataset,dataset_s],axis=1).fillna(method="ffill")
    dataset_s=dataset.iloc[:,5:]
    dataset=dataset.iloc[:,:5]
        
    if noise == 1:
        print("AGGIUNGO RUMORE")
        dataset=add_noise(dataset.open, dataset.high, dataset.low, dataset.close,dataset.volume, pct_noise)

    dataset = dataset.loc[IS:OOS]
    dataset_s = dataset_s.loc[IS:OOS]
    dataset["dayofweek"] = dataset.index.dayofweek
    dataset["day"] = dataset.index.day
    dataset["month"] = dataset.index.month
    dataset["year"] = dataset.index.year
    dataset["dayofyear"] = dataset.index.dayofyear
    dataset["quarter"] = dataset.index.quarter
    dataset["hour"] = dataset.index.hour
    dataset["minute"] = dataset.index.minute
    dataset["daily_open"] = daily_open(dataset,1)
    dataset["daily_high"] = daily_high(dataset,1)
    dataset["daily_low"] = daily_low(dataset,1)
    dataset["daily_close"] = daily_close(dataset,1)
    dataset["AVGPRICE"]=ta.AVGPRICE(dataset.open, dataset.high, dataset.low, dataset.close)
    dataset["MEDPRICE"]=ta.MEDPRICE(dataset.high, dataset.low)
    dataset["TYPPRICE"]=ta.TYPPRICE(dataset.high, dataset.low, dataset.close)
    dataset["WCLPRICE"]=ta.WCLPRICE(dataset.high, dataset.low, dataset.close)
    history=dataset.copy()
    
    print("Carico indicatori")
    import talib as ta
    applica_indicatori=apply_indicator(dataset_s)
    dataset=applica_indicatori[0]
    dataset_ind = dataset.iloc[:,5:].copy()


    rules=dataset.iloc[:,5:].T.values
    rule_formulas=np.arange(len(rules))
    
    history.tail(10)
    
    history.close.plot(figsize=(20,10),title=SIMBOLO+"_"+RESAMPLE_TF)
    print("Fatto")
    return dataset , dataset_ind , history , rules, rule_formulas

def target(direzione,_search,dataset):
    _search=_search
    close=history.close.copy()
    t_m=[]
    if direzione == "long":
        t_m=close-close.shift(_search)
        t_m=t_m.shift(-_search).fillna(0)
        t_m=np.array(t_m * bigpointvalue)
    else:
        t_m=close-close.shift(_search)
        t_m=t_m.shift(-_search).fillna(0)
        t_m=t_m*(-1)
        t_m=np.array(t_m * bigpointvalue)
    return t_m

def environment_second(simbolo,simbolo2,type_session,resample_tf,start_date,end_date):
               
    file_name = check_history_name(simbolo,dir_history)
    file_name2 = check_history_name(simbolo,dir_history) #simbolo2+".csv"

    IS = start_date
    OOS = end_date

    bigpointvalue=valori[valori.Physical==simbolo].BigPointValue.values[0]
    tick=valori[valori.Physical==simbolo].Tick.values[0]
    utc_offset = valori[valori.Physical==simbolo].Exchange_Time.values[0]
    custom_session_start = "0"+str(valori[valori.Physical==simbolo]["Custom Open"].values[0])
    custom_session_stop = str(valori[valori.Physical==simbolo]["Custom Close"].values[0])
    costi=valori[valori.Physical==simbolo].Cost.values[0]

    dataset , dataset_ind , history , rules, rule_formulas = carica_storico_second(file_name,file_name2,utc_offset,type_session,resample_tf,IS,OOS,custom_session_start,custom_session_stop,0,0)
    
    return dataset , dataset_ind , history , rules, rule_formulas , bigpointvalue , tick , utc_offset , custom_session_start , custom_session_stop , costi 

